This notebook demonstrates how to use PredictHQ’s Forecasts API. A sample demand dataset and configuration file is provided.

In [ ]:
# Install dependencies if not already installed
# !pip install -r requirements.txt

In [1]:
import os
import pandas as pd
import time
import json
import requests
import plotly.graph_objects as go

# Load Access Token

[Create a valid access token](https://docs.predicthq.com/getting-started/api-quickstart#create-an-access-token) if you do not already have one and load to access PredictHQ's APIs.

In [ ]:
PHQ_ACCESS_TOKEN = os.environ.get("PHQ_ACCESS_TOKEN") or "REPLACE_WITH_YOUR_ACCESS_TOKEN"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {PHQ_ACCESS_TOKEN}",
}
API_URL = "https://api.predicthq.com"

# Create a Saved Location

Create a Saved Location for the location you want to forecast. This allows you to reference the location easily and ensures consistency across forecasts. The ideal radius for the location can be calculated using the Suggested Radius API.

In [4]:
# Get location details
with open("data/sample_config.json", "r") as json_file:
    config = json.load(json_file)

print(config)

{'name': 'Sample Restaurant Location', 'lat': 51.50396, 'lon': 0.00476, 'industry': 'restaurants'}


In [5]:
# Get suggested radius
response = requests.get(
    url=f"{API_URL}/v1/suggested-radius/",
    headers=headers,
    params={
        "location.origin": f"{config['lat']},{config['lon']}",
        "industry": config["industry"],
        "radius_unit": "mi",
    },
)

data = response.json()
radius = data["radius"]
radius_unit = data["radius_unit"]

print(f"Suggested radius: {radius} {radius_unit}")

Suggested radius: 1.11 mi


In [6]:
# Create saved location
response = requests.post(
    url=f"{API_URL}/v1/saved-locations",
    headers=headers,
    data=json.dumps(
        {
            "name": config["name"],
            "geojson": {
                "type": "Feature",
                "properties": {"radius": radius, "radius_unit": radius_unit},
                "geometry": {
                    "type": "Point",
                    "coordinates": [config["lon"], config["lat"]],
                },
            },
        }
    ),
)

location_id = response.json()["location_id"]
print(f"Saved Location ID: {location_id}")

Saved Location ID: jJli0RvrFXONrMH8c0J4BA


# Define and Train a Forecast Model

In [7]:
# Define model
response = requests.post(
    url=f"{API_URL}/v1/forecasts/models",
    headers=headers,
    json={
        "name": f"{config['name']} Forecast",
        "location": {"saved_location_id": location_id},
        "algo": "phq-xgboost",
        "forecast_window": "7d",
        "demand_type": {
            "industry": config["industry"],
        },
    },
)

model_id = response.json()["model_id"]
print(f"Model ID: {model_id}")

Model ID: y-3UPg0xT3tQY66EfTbw8g


In [8]:
# Upload demand
sample_demand_df = pd.read_csv("data/sample_demand.csv")
sample_demand_json = sample_demand_df.to_json(orient="records")

response = requests.post(
    url=f"{API_URL}/v1/forecasts/models/{model_id}/demand",
    headers=headers,
    json={"demand": json.loads(sample_demand_json)},
)

print(f"Demand upload: {'Successful' if response.status_code == 201 else 'Failed'}")

Demand upload: Successful


In [9]:
# Train model
response = requests.post(
    url=f"{API_URL}/v1/forecasts/models/{model_id}/train",
    headers=headers,
)

print(f"Model training: {'Triggered' if response.status_code == 204 else 'Failed'}")

Model training: Triggered


# Check Model Status

The model training may take up to a few minutes. Make sure the model is `ready` before proceeding.

In [10]:
while True:
    response = requests.get(
        url=f"{API_URL}/v1/forecasts/models/{model_id}",
        headers=headers,
    )
    if response.status_code != 200:
        raise Exception(f"Failed to get model status: {response.status_code}, {response.text}")
    
    model_status = response.json()["model"]["readiness"]["status"]
    if model_status == "ready":
        print("Model is ready!")
        break
    if model_status == "failed":
        raise Exception("Model training failed")

    print(f"Model is {model_status}. Checking again in 30 seconds...")
    time.sleep(30)

Model is preparing. Checking again in 30 seconds...
Model is training. Checking again in 30 seconds...
Model is ready!


# Evaluate Forecast Model

Use evaluation metrics such as MAPE to compare the model performance to other models, benchmarks, etc. In this example, the benchmark model had a MAPE of 13.05%.

In [11]:
# Get evaluation results
response = requests.get(
    url=f"{API_URL}/v1/forecasts/models/{model_id}",
    headers=headers,
)

metrics = response.json()['model']['metrics']
print(f"Evaluation metrics: {json.dumps(metrics, indent=2)}")

Evaluation metrics: {
  "accuracy": {
    "mape": 9.28,
    "mae": 1751.09,
    "rmse": 2373.64
  },
  "demand_data": {
    "date_range": {
      "start": "2023-02-03",
      "end": "2023-08-02"
    }
  },
  "training_data": {
    "date_range": {
      "start": "2023-02-03",
      "end": "2023-08-02"
    },
    "missing_pct": 0.0,
    "missing_dates": []
  }
}


# Generate Forecasts

In [12]:
# Get forecast
response = requests.get(
    url=f"{API_URL}/v1/forecasts/models/{model_id}/forecast",
    headers=headers,
    params={"date.gt": "2023-08-02"},
)

results = response.json()["results"]
forecasts_df = pd.DataFrame(results)

## Visualise results

In [13]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=sample_demand_df["date"],
        y=sample_demand_df["demand"],
        mode="lines+markers",
        name="Actual",
    )
)
fig.add_trace(
    go.Scatter(
        x=forecasts_df["date"],
        y=forecasts_df["forecast"],
        mode="lines+markers",
        name="Forecast",
    )
)
fig.update_layout(
    title="Forecasts for the next 7 days",
    xaxis_title="Date",
    yaxis_title="Demand",
)

fig